# Uncertainty propagation applied to ordinary differential equation model using DiffEqUncertainty.jl
Simon Frost (@sdwfrost), 2022-03-02

## Introduction

If we are interested in investigating how uncertainty in parameters propagates to the mean, variance, etc. of the output, we can use the uncertainty propagation routines in `DiffEqUncertainty.jl`. As in the [`MonteCarloMeasurements.jl` example](https://github.com/epirecipes/sir-julia/blob/master/markdown/ode_montecarlomeasurements/ode_montecarlomeasurements.md), we can define uncertain initial conditions and parameter values using distributions, although these are assumed to be independent. Two algorithms are provided; a simple Monte Carlo method, and a [Koopman expectation method](https://arxiv.org/abs/2008.08737).

## Libraries

In [ ]:
using OrdinaryDiffEq
using Distributions
using DiffEqUncertainty
using Plots

## Helper functions

The `centralmoment` function in `DiffEqUncertainty.jl` currently only works for the `Koopman` method. The following fix taken from [this pull request](https://github.com/SciML/DiffEqUncertainty.jl/pull/29) is a workaround, at least for this example.

In [ ]:
function my_centralmoment(n::Int, g::Function, args...; kwargs...)
    if n < 2 return Float64[] end
    # Compute the expectations of g, g^2, ..., g^n
    sol = expectation(x -> [g(x)^i for i in 1:n], args...; nout = n, kwargs...)
    exp_set = sol[:]
    mu_g = popfirst!(exp_set)
    # Combine according to binomial expansion
    const_term(n) = (-1)^(n-1) * (n-1) * mu_g^n
    binom_term(n, k, mu, exp_gi) = binomial(n, k) * (-mu)^(n - k) * exp_gi
    binom_sum = function (exp_vals)
        m = length(exp_vals) + 1
        sum([binom_term(m, k + 1, mu_g, v) for (k,v) in enumerate(exp_vals)]) + const_term(m)
    end
    return [zero(exp_set[1]), [binom_sum(exp_set[1:i]) for i in 1:length(exp_set)]...]
end

## Transitions

In [ ]:
function sir_ode!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    N = S+I+R
    @inbounds begin
        du[1] = -β*c*I/N*S
        du[2] = β*c*I/N*S - γ*I
        du[3] = γ*I
    end
    nothing
end;

## Time domain

In [ ]:
δt = 1.0
tmax = 40.0
tspan = (0.0,tmax);
t = 0:δt:tmax;

## Initial conditions and parameter values

In [ ]:
u0 = [990.0,10.0,0.0]
p = [0.05,10,0.25]
prob_ode = ODEProblem(sir_ode!,u0,tspan,p);

We will assume that the initial conditions are fixed, but that the parameter values, `β`, `c`, and `γ` are uncertain. We can use arbitrary distributions here, but for simplicity and for a closer comparison with methods that simply take a lower and an upper bound, we use `Uniform` distributions.

In [ ]:
p_dist = [Uniform(0.01,0.1),
     Uniform(5,20.0),
     Uniform(0.1,1.0)];

## Running the model

We need to define output functions that take the solution of an ODE as input; two are used, one that outputs the entire solution for the time points `t` defined above, the second that outputs the `i`th variable at time `x`.

In [ ]:
g(sol) = sol(t)
g(sol,x,i) = sol(x)[i];

### Monte Carlo method

In [ ]:
n_samples = 1000
sol_ode_mean_mc = expectation(g, prob_ode, u0, p_dist, MonteCarlo(), Tsit5(); trajectories = n_samples)
sol_ode_mean_mc = Array(sol_ode_mean_mc)'

`centralmoment` does not currently support vector results (see the issue [here](https://github.com/SciML/DiffEqUncertainty.jl/issues/22)), so as a workaround, we cycle through states and times in order to get the standard deviations of the state variables at each time point.

In [ ]:
sol_ode_std_mc = [[sqrt(my_centralmoment(2, (sol) -> g(sol,x,i), prob_ode, u0, p_dist, MonteCarlo(), Tsit5(); trajectories = n_samples)[2]) for x in t] for i in 1:3]
sol_ode_std_mc = hcat(sol_ode_std_mc...)

### Koopman expectation method

In [ ]:
sol_ode_mean_k = expectation(g, prob_ode, u0, p_dist, Koopman(), Tsit5())
sol_ode_mean_k = Array(sol_ode_mean_k)'

In [ ]:
sol_ode_std_k = [[sqrt(my_centralmoment(2, (sol) -> g(sol,x,i), prob_ode, u0, p_dist, Koopman(), Tsit5())[2]) for x in t] for i in 1:3]
sol_ode_std_k = hcat(sol_ode_std_k...)

## Plotting

In [ ]:
l = @layout [a b]
pl1 = plot(t,
     sol_ode_mean_mc,
     ribbon=sol_ode_std_mc,
     fillalpha=0.15,
     label=["S" "I" "R"],
     xlabel="Time",
     ylabel="Number",
     title="Monte Carlo")
pl2 = plot(t,
     sol_ode_mean_k,
     ribbon=sol_ode_std_k,
     fillalpha=0.15,
     label=["S" "I" "R"],
     xlabel="Time",
     ylabel="Number",
     title="Koopman")
plot(pl1,pl2,layout=l)

The results from `MonteCarlo()` and `Koopman()` are similar; however that the underlying distributions are far from normal, so the above ribbon plot is a bit misleading (for example, the ribbons go below 0 and above `N=1000`, the bounds of the population). In addition, it is difficult to accommodate correlation between the uncertain inputs; hence, the uncertainty in the initial condition used in the [`MonteCarloMeasurements.jl` example](https://github.com/epirecipes/sir-julia/blob/master/markdown/ode_montecarlomeasurements/ode_montecarlomeasurements.md) was not considered here.